In [ ]:
import torch
from transformers import pipeline
from huggingface_hub import login
# Load model directly
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import re

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
token = os.getenv("HUGGINGFACE_TOKEN")
os.environ["HUGGINGFACE_TOKEN"] = os.getenv("HUGGINGFACE_TOKEN")

In [ ]:
!huggingface-cli login --token "$HUGGINGFACE_TOKEN"

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `samuel` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `samuel`


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_excel('/content/drive/MyDrive/Agent_LLM_paper/data.xlsx')

In [ ]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
def feature_extraction(drug_name, scraping, review) :
  messages = [
      {"role": "system", "content": """You are medical text evaluator specializing in drug reviews"""},
      {"role": "user", "content": f"""
      Given the following text about the drug {drug_name} from *Drugs.com*, identify the following characteristics and rate them on a scale of -5 to 5:

      1. **Perceived effectiveness**: Effectiveness rating based on opinions and therapeutic indication(A score of -5 indicates perceived ineffectiveness, while +5 reflects very high perceived effectiveness)
      2. **Speed ​​of action**: Release time after administration(A score of -5 signifies a very long relief time, while +5 indicates almost immediate relief after administration)
      3. **Severity of side effects**: Level of severity of side effects(A score of -5 indicates frequent and severe side effects, while +5 means mild and rare side effects)
      4. **Perceived tolerance**: Ability of the user to tolerate the treatment in the long term(A score of -5 signifies poor long-term tolerance, while +5 reflects excellent tolerance for extended use)
      5. **Convenience of use**: Ease of use based on formulation type and frequency of administration(A score of -5 indicates difficult usage (e.g., high frequency or complex instructions), while +5 signifies high convenience in use)
      6. **Perceived cost**: Evaluation of the cost (or mention of associated terms)(A score of -5 indicates a very high perceived cost (or unaffordability), while +5 means a very affordable perceived cost)
      7. **Overall satisfaction**: Indicator of general user satisfaction(A score of -5 indicates general dissatisfaction, while +5 reflects high satisfaction among users)

      Here is the output expected:
      "@<Perceived_effectiveness>@<Speed_of_action>@<Severity_of_side_effects>@<Perceived_tolerance>@<Convenience_of_use>@<Perceived_cost>@<Overall_satisfaction>@"

      For each characteristic, returns a qualitative evaluation followed by a score between -5 (very negative) and 5 (very positive), be objective and critic
      Here is the web scraping result :
          {scraping}

      Also take into account the reviews of people who have used this medication :
          {review}

      Don't say anything, just give values output as follows :
      "@<Perceived_effectiveness>@<Speed_of_action>@<Severity_of_side_effects>@<Perceived_tolerance>@<Convenience_of_use>@<Perceived_cost>@<Overall_satisfaction>@"
      Your rating must not be higher than 5 nor lower than -5
      """},]

  outputs = pipe(
  messages,
  max_new_tokens=256,)
  result =  outputs[0]["generated_text"][-1]['content']

  pattern = r"@(-?\d+)"
  matches = re.findall(pattern, result)
  values = [int(match) for match in matches]
  if values :
      return values
  return None

In [ ]:
#We compute manually per batch
data['feature extraction'] = data.progress_apply(lambda row: feature_extraction(row['drugName'], row['scraping'], row['review_x']), axis=1)

Streaming output truncated to the last 5000 lines.
  0%|          | 11/5000 [00:18<2:07:12,  1.53s/it]You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
100%|██████████| 5000/5000 [6:14:49<00:00,  4.50s/it]
<ipython-input-8-264ba4a5351b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['feature extraction'] = df.progress_apply(lambda row: feature_extraction(row['drugName'], row['scraping'], row['review_x']), axis=1)


In [ ]:
data = data[['Unnamed: 0', 'drugName', 'feature extraction']]

In [ ]:
data.to_excel("/content/drive/MyDrive/Agent_LLM_paper/data.xlsx", index=False)